In [6]:
import os
import gc
import torch
import timm
import requests
import numpy as np
from PIL import Image
import pandas as pd
from tqdm import tqdm
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets,transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split,DataLoader, Dataset
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

In [20]:
class ImageDataset(Dataset):
    def __init__(self, inmeyok_path, iskemi_path, kanama_path,normal,stroke ,transform=None,sadece_teknofest=False):
        self.images = []
        self.transform = transform

        valid_extensions = (".png", ".jpg", ".jpeg")
        for imgs in os.listdir(inmeyok_path):
            self.images.append((os.path.join(inmeyok_path, imgs), 0))
        for imgs in os.listdir(iskemi_path):
            
            self.images.append((os.path.join(iskemi_path, imgs), 1))

        for imgs in os.listdir(kanama_path):  
            self.images.append((os.path.join(kanama_path, imgs), 1))
        if sadece_teknofest==False:
            for imgs in os.listdir(normal):
                self.images.append((os.path.join(normal, imgs), 0))
    
            for imgs in os.listdir(stroke):
                self.images.append((os.path.join(stroke, imgs), 1))
        print(f"Toplam {len(self.images)} görüntü yüklendi.")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        image = Image.open(img_path).convert("L") 
          
        return image, label
        
class TestSet(Dataset):
    def __init__(self,normal,stroke,transform):
        self.images = []
        self.transform = transform

        valid_extensions = (".png", ".jpg", ".jpeg")
        for imgs in os.listdir(normal):
                self.images.append((os.path.join(normal, imgs), 0))
    
        for imgs in os.listdir(stroke):
            self.images.append((os.path.join(stroke, imgs), 1))
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        image = Image.open(img_path).convert("L") 
        image = self.transform(image)

        return image, torch.tensor([label], dtype=torch.float32)
        
       
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class Transform(Dataset):
    def __init__(self, base_dataset, transform):
        self.base_dataset = base_dataset
        self.transform = transform

    def __len__(self):
        return len(self.base_dataset)

    def __getitem__(self, idx):
        image, label = self.base_dataset[idx]
        if self.transform:
            image = self.transform(image)
        return image, label
        
class StrokeDataset(Dataset):
    def __init__(self, image_dir, label_path, transform):
        self.image_dir = image_dir
        self.transform = transform
        self.labels = {}

        # Etiketleri yükleme
        with open(label_path, "r", encoding="utf-8") as f:
            lines = f.readlines()[1:]  # İlk satırı (başlık) atlıyoruz
            for line in lines:
                parts = line.strip().split("\t")  # Tab ile ayrılmış sütunlar
                if len(parts) == 2:
                    image_id, label = parts
                    self.labels[image_id] = int(label)

        self.image_names = sorted(self.labels.keys())

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_id = self.image_names[idx]
        img_path = os.path.join(self.image_dir, f"{image_id}.png")  # PNG uzantısını ekliyoruz
        image = Image.open(img_path).convert("L")

        label = self.labels[image_id]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor([label], dtype=torch.float32)

Using device: cuda


In [21]:
file_path = "C:\\Users\\Mehmet\\Desktop\\teknofest\\İNME VERİ SETİ" 
inmeyok_path = file_path+"\\İnme Yok"
iskemi_path = file_path+"\\İskemi\\PNG"
kanama_path = file_path+"\\Kanama\\PNG"                                      #TEKNOFEST VERİLERİ
augemented_path = "C:\\Users\\Mehmet\\Desktop\\teknofest\\Brain_Data_Organised"  #İNTERNETTEN BULUNAN  VERİSETİ
test_path = "C:\\Users\\Mehmet\\Desktop\\teknofest\\stroke.v1i.folder\\test" #Test için ekstra indirdiğimiz veri seti
normal=augemented_path+"\\Normal"
stroke=augemented_path+"\\Stroke"
normal2 = test_path +"\\Normal"
stroke2 = test_path + "\\Stroke"
test_data_path = file_path+"\\Test Veri Seti_1\\PNG"
test_label_path= file_path+"\\Test Veri Seti_1\\ASAMA1_Cevaplar.txt"        #TEKNOFEST TEST VERİSİ
image_size=256
batch_size=32

test_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  
])

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Yatay çevirme (Görsellerin %50'sini ters çevir)
    transforms.RandomRotation(10),      # 10 dereceye kadar döndürme
    transforms.RandomResizedCrop(256, scale=(0.9, 1.0)),  # Resmi rastgele %90-%100 arasında kırpma
    transforms.ColorJitter(brightness=0.1, contrast=0.1),  # Hafif renk değişimi
    transforms.RandomAffine(5, shear=5),  # Az miktarda afine dönüşüm (Eğim ve döndürme)
    transforms.ToTensor(),
    transforms.Resize((image_size, image_size)),# Tensor dönüşümü
    transforms.Normalize((0.5,), (0.5,))  # Normalizasyon (CT görüntüleri için uygun olabilir)
])

train_val_dataset = ImageDataset(inmeyok_path, iskemi_path, kanama_path,normal,stroke,sadece_teknofest=False) #teknofest verisi
train_size = int(0.8 * len(train_val_dataset))  
val_size = len(train_val_dataset) - train_size 

train_dataset, val_dataset = random_split(train_val_dataset, [train_size, val_size])

train_dataset = Transform(train_dataset, train_transform)
val_dataset = Transform(val_dataset, test_transform)
test_dataset = StrokeDataset(test_data_path, test_label_path,test_transform)
test2_dataset = TestSet(normal2,stroke2,test_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,pin_memory=True, num_workers=0)
test2_loader = DataLoader(test2_dataset, batch_size=batch_size, shuffle=False,pin_memory=True, num_workers=0)

Toplam 9275 görüntü yüklendi.


In [14]:
def train_model(model, epochs, optimizer, criterion, train_loader,val_loader, device,early_stopping_patience):
    scaler = torch.amp.GradScaler()
    early_stopping_patience =early_stopping_patience  # 5 epoch boyunca iyileşmezse durdur
    best_acc = 0.0
    epochs_without_improvement = 0
    best_model_weights = None
    for epoch in range(epochs):
        # Eğitim aşaması
        model.train()
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0
        
        train_pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs} - Training")
        for step, train_batch in train_pbar:
            
            images, labels = train_batch
            images = images.to(device, dtype=torch.float32)
            labels = labels.unsqueeze(1).float().to(device)
            #labels = labels.to(device)
          
            optimizer.zero_grad()  # Optimizer'ı sıfırla
            
            with torch.amp.autocast(device_type='cuda'):
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()  # Gradients hesapla
            scaler.step(optimizer)  # Optimizer adımını yap
            scaler.update()  # GradScaler'ı güncelle
            

            running_loss += loss.item()

            # Predicted sınıf
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)

            #if step % 10 == 0:
                #print(f"Step {step}, Train_Loss: {running_loss / (step + 1):.4f}")

        # Eğitim loss ve accuracy hesapla
        train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct_preds / total_preds

        # Doğrulama aşaması
        model.eval()
        val_loss = 0.0
        val_correct_preds = 0
        val_total_preds = 0
        val_pbar = tqdm(enumerate(val_loader), total=len(val_loader), desc=f"Epoch {epoch+1}/{epochs} - Validation")
        with torch.no_grad():
            for step, val_batch in val_pbar:  # enumerate ekledik
                images, labels = val_batch
                images = images.to(device, dtype=torch.float32)
                labels = labels.unsqueeze(1).float().to(device)
                labels = labels.to(device)

                with torch.amp.autocast(device_type='cuda'):
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    
               
                val_loss += loss.item()

                # Predicted sınıf
                predicted = (torch.sigmoid(outputs) > 0.2).float()
                val_correct_preds += (predicted == labels).sum().item()
                val_total_preds += labels.size(0)

                #if step % 10 == 0:
                    #print(f"Step {step}, Val_Loss: {val_loss / (step + 1):.4f}")

                # Bellek temizleme
                del images, labels, outputs
                torch.cuda.empty_cache()

        # Doğrulama loss ve accuracy hesapla
        val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * val_correct_preds / val_total_preds

        # Epoch sonunda eğitim ve doğrulama sonuçlarını yazdır
        print(f"Epoch {epoch + 1}/{epochs}, "
              f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
        if val_accuracy > best_acc:
            print(f"Yeni en iyi doğruluk bulundu: {val_accuracy:.4f}")
            best_acc = val_accuracy
            best_model_weights = model.state_dict()
            epochs_without_improvement = 0  # iyileşme olduğu için sayaç sıfırlanır
        else:
            epochs_without_improvement += 1
    
        # Early stopping: Eğer 5 epoch boyunca iyileşme olmazsa, eğitim durdurulacak
        if epochs_without_improvement >= early_stopping_patience:
            print("Early stopping uygulandı, eğitim durduruluyor.")
            break
    
    # Eğitim sonrasında en iyi modelin ağırlıklarını kaydedin
    if best_model_weights:
        print("En iyi modelin ağırlıkları kaydediliyor.")
        torch.save({
                'model_state_dict': model.state_dict(),
                'criterion_state_dict': criterion.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                },"best_weights.pth")
def test_model(model,test_loader,criterion,device):
    model.eval()  
    
    test_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    result=[]
    
    with torch.no_grad():  
        for inputs, labels in test_loader:
            labels = labels.to(device)
            inputs = inputs.to(device, dtype=torch.float32)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            # Predicted class with highest score
            predicted = (torch.sigmoid(outputs) > 0.2).float()
            #_, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
            for true_label, pred_label in zip(labels.cpu(), predicted.cpu()):
                result.append([true_label.item(), pred_label.item()])
    # Ortalama test kaybı
    test_loss /= len(test_loader)
    
    # Test doğruluğu (accuracy)
    test_accuracy = 100 * correct_predictions / total_predictions
    
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')
    
    return result

In [15]:
def create_resnet_model(num_classes=2, freeze_layers=False):
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
    model.conv1 = nn.Conv2d(1,64,kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    if freeze_layers:
        for param in model.parameters():
            param.requires_grad = False
    in_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(in_features, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, num_classes))
    return model
def create_densenet121_model(num_classes=2, freeze_layers=False):
    model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
    model.features.conv0 = nn.Conv2d(1,64,kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    if freeze_layers:
        for param in model.parameters():
            param.requires_grad = False
    in_features = model.classifier.in_features
    model.classifier = nn.Sequential(
        nn.Linear(in_features, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, num_classes))
    return model
def create_efficientnet_model(num_classes=2, freeze_layers=False):
    # EfficientNet-B0 modelini yükle
    model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

    # 1-kanallı (grayscale) girişe uyum sağla
    model.features[0][0] = nn.Conv2d(
        1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
    )

    if freeze_layers:
        for param in model.parameters():
            param.requires_grad = False

    in_features = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Linear(in_features, 512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, num_classes)
    )

    return model

class HybridModel(nn.Module):
    def __init__(self, base_model,num_classes,embed_dim,num_layers,num_heads,dropout):
        super(HybridModel, self).__init__()
        
       
        
        self.base_model = base_model #efficient net
        self.base_model = nn.Sequential(*list(self.base_model.children())[:-2])
        self.efficient = self.base_model

        self.projector = nn.Linear(1280, embed_dim)
        self.pos_embed = nn.Parameter(torch.randn(1, 64, embed_dim))  # 8x8 = 64 patch

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=embed_dim * 4,
            dropout=dropout,
            batch_first=True  # [B, S, E]
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.classifier = nn.Linear(embed_dim, num_classes)
    def forward(self, x):
        
        x = self.efficient(x)
        x = x.flatten(2).permute(0, 2, 1)       # [B, 64, 1280]
        x = self.projector(x)                   # [B, 64, embed_dim]
        x = x + self.pos_embed                  # Positional Embedding ekle
        x = self.transformer(x)                 # [B, 64, embed_dim]

        # Ortalama pooling 
        x = x.mean(dim=1)                       # [B, embed_dim]

        return self.classifier(x)

In [29]:
model = create_resnet_model(num_classes=1, freeze_layers=False)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss() 
epochs=50

In [32]:
train_model(model,epochs,optimizer,criterion, train_loader,val_loader,device)

Epoch 1/50 - Validation: 100%|█████████████████████████████████████████████████████████| 29/29 [00:10<00:00,  2.68it/s]


Epoch 1/50, Train Loss: 0.4177, Train Accuracy: 80.53%, Val Loss: 0.3737, Val Accuracy: 84.42%
Yeni en iyi doğruluk bulundu: 84.4205


Epoch 2/50 - Validation: 100%|█████████████████████████████████████████████████████████| 29/29 [00:10<00:00,  2.83it/s]


Epoch 2/50, Train Loss: 0.2811, Train Accuracy: 88.14%, Val Loss: 0.2270, Val Accuracy: 90.57%
Yeni en iyi doğruluk bulundu: 90.5660


Epoch 3/50 - Validation: 100%|█████████████████████████████████████████████████████████| 29/29 [00:09<00:00,  2.98it/s]


...

Epoch 17/50, Train Loss: 0.0396, Train Accuracy: 98.49%, Val Loss: 0.1519, Val Accuracy: 95.31%


Epoch 18/50 - Validation: 100%|████████████████████████████████████████████████████████| 29/29 [00:10<00:00,  2.74it/s]


Epoch 18/50, Train Loss: 0.0349, Train Accuracy: 98.75%, Val Loss: 0.1301, Val Accuracy: 95.85%


Epoch 19/50 - Validation: 100%|████████████████████████████████████████████████████████| 29/29 [00:10<00:00,  2.75it/s]

Epoch 19/50, Train Loss: 0.0325, Train Accuracy: 98.85%, Val Loss: 0.1105, Val Accuracy: 96.39%
Early stopping uygulandı, eğitim durduruluyor.
En iyi modelin ağırlıkları kaydediliyor.


In [18]:
model = create_densenet121_model(num_classes=1, freeze_layers=False).to(device)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss() 
epochs=50

In [19]:
train_model(model,epochs,optimizer,criterion, train_loader,val_loader,device)

Epoch 1/50 - Validation: 100%|█████████████████████████████████████████████████████████| 29/29 [01:10<00:00,  2.44s/it]


Epoch 1/50, Train Loss: 0.4365, Train Accuracy: 79.27%, Val Loss: 0.3758, Val Accuracy: 82.64%
Yeni en iyi doğruluk bulundu: 82.6415


Epoch 2/50 - Validation: 100%|█████████████████████████████████████████████████████████| 29/29 [01:08<00:00,  2.38s/it]


Epoch 2/50, Train Loss: 0.2723, Train Accuracy: 88.46%, Val Loss: 0.2535, Val Accuracy: 89.27%
Yeni en iyi doğruluk bulundu: 89.2722


Epoch 3/50 - Validation: 100%|█████████████████████████████████████████████████████████| 29/29 [01:07<00:00,  2.34s/it]


...

Epoch 17/50 - Validation: 100%|████████████████████████████████████████████████████████| 29/29 [00:10<00:00,  2.69it/s]


Epoch 17/50, Train Loss: 0.0319, Train Accuracy: 98.63%, Val Loss: 0.1335, Val Accuracy: 95.74%


Epoch 18/50 - Validation: 100%|████████████████████████████████████████████████████████| 29/29 [00:10<00:00,  2.67it/s]

Epoch 18/50, Train Loss: 0.0393, Train Accuracy: 98.77%, Val Loss: 0.1719, Val Accuracy: 94.77%
Early stopping uygulandı, eğitim durduruluyor.
En iyi modelin ağırlıkları kaydediliyor.


NameError: name 'model_next' is not defined

In [31]:
model = create_efficientnet_model(num_classes=1, freeze_layers=False).to(device)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss() 
epochs=50

In [95]:
train_model(model,epochs,optimizer,criterion, train_loader,val_loader,device)

NameError: name 'train_model' is not defined

In [15]:
efficient = create_efficientnet_model(num_classes=1, freeze_layers=False).to(device)
num_classes=1
embed_dim=512
num_layers=4
num_heads=8
dropout=0.2
model = HybridModel(base_model=efficient,num_classes=num_classes,embed_dim=embed_dim,num_layers=num_layers,num_heads=num_heads,dropout=dropout)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()
epochs=50

In [15]:
train_model(model,epochs,optimizer,criterion, train_loader,val_loader,device)

Epoch 1/50 - Validation: 100%|█████████████████████████████████████████████████████████| 58/58 [00:08<00:00,  7.00it/s]


Epoch 1/50, Train Loss: 0.4651, Train Accuracy: 78.07%, Val Loss: 0.3545, Val Accuracy: 85.55%
Yeni en iyi doğruluk bulundu: 85.5526


Epoch 2/50 - Validation: 100%|█████████████████████████████████████████████████████████| 58/58 [00:08<00:00,  6.89it/s]


Epoch 2/50, Train Loss: 0.3081, Train Accuracy: 86.94%, Val Loss: 0.2688, Val Accuracy: 88.79%
Yeni en iyi doğruluk bulundu: 88.7871


Epoch 3/50 - Validation: 100%|█████████████████████████████████████████████████████████| 58/58 [00:08<00:00,  6.88it/s]


...

Epoch 18/50, Train Loss: 0.0421, Train Accuracy: 98.45%, Val Loss: 0.1334, Val Accuracy: 96.71%


Epoch 19/50 - Validation: 100%|████████████████████████████████████████████████████████| 58/58 [00:08<00:00,  6.91it/s]


Epoch 19/50, Train Loss: 0.0365, Train Accuracy: 98.65%, Val Loss: 0.1224, Val Accuracy: 96.71%


Epoch 20/50 - Validation: 100%|████████████████████████████████████████████████████████| 58/58 [00:08<00:00,  6.91it/s]


Epoch 20/50, Train Loss: 0.0371, Train Accuracy: 98.71%, Val Loss: 0.0949, Val Accuracy: 97.41%
Early stopping uygulandı, eğitim durduruluyor.
En iyi modelin ağırlıkları kaydediliyor.


In [19]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

In [20]:
train_model(model,epochs,optimizer,criterion, train_loader,val_loader,device)

Epoch 1/50 - Validation: 100%|█████████████████████████████████████████████████████████| 58/58 [00:08<00:00,  6.94it/s]


Epoch 1/50, Train Loss: 0.0256, Train Accuracy: 98.95%, Val Loss: 0.0929, Val Accuracy: 97.84%
Yeni en iyi doğruluk bulundu: 97.8437


Epoch 2/50 - Validation: 100%|█████████████████████████████████████████████████████████| 58/58 [00:08<00:00,  6.86it/s]


Epoch 2/50, Train Loss: 0.0193, Train Accuracy: 99.33%, Val Loss: 0.0909, Val Accuracy: 97.79%


Epoch 3/50 - Validation: 100%|█████████████████████████████████████████████████████████| 58/58 [00:08<00:00,  6.95it/s]


...

Epoch 9/50, Train Loss: 0.0085, Train Accuracy: 99.68%, Val Loss: 0.0946, Val Accuracy: 98.01%


Epoch 10/50 - Validation: 100%|████████████████████████████████████████████████████████| 58/58 [00:08<00:00,  6.91it/s]


Epoch 10/50, Train Loss: 0.0099, Train Accuracy: 99.64%, Val Loss: 0.0917, Val Accuracy: 98.06%


Epoch 11/50 - Validation: 100%|████████████████████████████████████████████████████████| 58/58 [00:08<00:00,  7.01it/s]


Epoch 11/50, Train Loss: 0.0094, Train Accuracy: 99.72%, Val Loss: 0.0948, Val Accuracy: 97.90%
Early stopping uygulandı, eğitim durduruluyor.
En iyi modelin ağırlıkları kaydediliyor.


In [23]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.000002)
train_model(model,epochs,optimizer,criterion, train_loader,val_loader,device)

Epoch 1/50 - Validation: 100%|█████████████████████████████████████████████████████████| 58/58 [00:08<00:00,  6.56it/s]


Epoch 1/50, Train Loss: 0.0101, Train Accuracy: 99.69%, Val Loss: 0.0877, Val Accuracy: 98.17%
Yeni en iyi doğruluk bulundu: 98.1671


Epoch 2/50 - Validation: 100%|█████████████████████████████████████████████████████████| 58/58 [00:08<00:00,  7.00it/s]


Epoch 2/50, Train Loss: 0.0070, Train Accuracy: 99.76%, Val Loss: 0.0919, Val Accuracy: 98.11%


Epoch 3/50 - Validation: 100%|█████████████████████████████████████████████████████████| 58/58 [00:08<00:00,  6.64it/s]


...

Epoch 8/50, Train Loss: 0.0075, Train Accuracy: 99.76%, Val Loss: 0.0938, Val Accuracy: 98.06%


Epoch 9/50 - Validation: 100%|█████████████████████████████████████████████████████████| 58/58 [00:08<00:00,  6.99it/s]


Epoch 9/50, Train Loss: 0.0066, Train Accuracy: 99.80%, Val Loss: 0.0951, Val Accuracy: 98.17%


Epoch 10/50 - Validation: 100%|████████████████████████████████████████████████████████| 58/58 [00:08<00:00,  6.88it/s]


Epoch 10/50, Train Loss: 0.0078, Train Accuracy: 99.77%, Val Loss: 0.0921, Val Accuracy: 98.11%
Early stopping uygulandı, eğitim durduruluyor.
En iyi modelin ağırlıkları kaydediliyor.


In [25]:
def get_average_gradients(model):
    grad_norms = {}
    for name, param in model.named_parameters():
        if param.grad is not None:
            module_name = name.split('.')[0]
            grad_norms.setdefault(module_name, []).append(param.grad.norm().item())

    # Ortalamaları al
    grad_means = {k: sum(v)/len(v) for k, v in grad_norms.items()}
    return grad_means

In [26]:
train_model(model,epochs,optimizer,criterion, train_loader,val_loader,device)

Epoch 1/50 - Training:   0%|▎                                                          | 1/232 [00:00<02:01,  1.90it/s]

{'pos_embed': 47.207679748535156, 'base_model': 1156.8834515523156, 'projector': 1307.5956192016602, 'transformer': 122.26741840442021, 'classifier': 1149.6832275390625}


Epoch 1/50 - Training:   1%|▌                                                          | 2/232 [00:00<01:27,  2.64it/s]

{'pos_embed': 112.90938568115234, 'base_model': nan, 'projector': 2059.4961700439453, 'transformer': 148.92225770155588, 'classifier': 1311.5400390625}


Epoch 1/50 - Training:   1%|▊                                                          | 3/232 [00:01<01:17,  2.95it/s]

...

{'pos_embed': 1.1375718116760254, 'base_model': 23.995135597232025, 'projector': 27.1991548538208, 'transformer': 2.42711449166139, 'classifier': 25.559768676757812}


Epoch 11/50 - Training: 100%|████████████████████████████████████████████████████████| 232/232 [01:12<00:00,  3.21it/s]


{'pos_embed': 0.21543419361114502, 'base_model': 2.537936450496681, 'projector': 3.541570544242859, 'transformer': 0.4663086603783692, 'classifier': 9.043315887451172}


Epoch 11/50 - Validation: 100%|████████████████████████████████████████████████████████| 58/58 [00:09<00:00,  6.27it/s]


Epoch 11/50, Train Loss: 0.0050, Train Accuracy: 99.81%, Val Loss: 0.0934, Val Accuracy: 98.27%
Early stopping uygulandı, eğitim durduruluyor.
En iyi modelin ağırlıkları kaydediliyor.


In [29]:
for param in model.transformer.parameters():
    param.requires_grad = False

In [17]:
optimizer = torch.optim.Adam([
    {'params': model.base_model.parameters(), 'lr': 1e-6},
    {'params': model.projector.parameters(), 'lr': 1e-6},
    {'params': model.classifier.parameters(), 'lr': 2e-6},
])

In [ ]:
train_model(model,epochs,optimizer,criterion, train_loader,val_loader,device)

In [36]:
checkpoint=torch.load("best_weights.pth", map_location=device)

In [37]:
efficient = create_efficientnet_model(num_classes=1, freeze_layers=False).to(device)
num_classes=1
embed_dim=512
num_layers=4
num_heads=8
dropout=0.4
model = HybridModel(base_model=efficient,num_classes=num_classes,embed_dim=embed_dim,num_layers=num_layers,num_heads=num_heads,dropout=dropout)
model.to(device)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6, weight_decay=1e-5)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
criterion = torch.nn.BCEWithLogitsLoss()
epochs=50

In [13]:
neg = sum(cm[0])
pos = sum(cm[1])
pos_weight = torch.tensor([neg / pos])
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
criterion.to(device)

BCEWithLogitsLoss()

In [14]:
train_model(model,epochs,optimizer,criterion, train_loader,val_loader,device,5)

Epoch 1/50 - Validation: 100%|█████████████████████████████████████████████████████████| 58/58 [00:11<00:00,  5.10it/s]


Epoch 1/50, Train Loss: 0.1389, Train Accuracy: 96.93%, Val Loss: 0.0120, Val Accuracy: 99.78%
Yeni en iyi doğruluk bulundu: 99.7844


Epoch 2/50 - Validation: 100%|█████████████████████████████████████████████████████████| 58/58 [00:08<00:00,  6.91it/s]


Epoch 2/50, Train Loss: 0.1335, Train Accuracy: 97.14%, Val Loss: 0.0132, Val Accuracy: 99.73%


Epoch 3/50 - Validation: 100%|█████████████████████████████████████████████████████████| 58/58 [00:08<00:00,  6.94it/s]


...

Epoch 4/50, Train Loss: 0.1085, Train Accuracy: 97.67%, Val Loss: 0.0152, Val Accuracy: 99.73%


Epoch 5/50 - Validation: 100%|█████████████████████████████████████████████████████████| 58/58 [00:08<00:00,  6.88it/s]


Epoch 5/50, Train Loss: 0.1068, Train Accuracy: 97.57%, Val Loss: 0.0133, Val Accuracy: 99.68%


Epoch 6/50 - Validation: 100%|█████████████████████████████████████████████████████████| 58/58 [00:08<00:00,  6.93it/s]


Epoch 6/50, Train Loss: 0.1042, Train Accuracy: 97.45%, Val Loss: 0.0126, Val Accuracy: 99.68%
Early stopping uygulandı, eğitim durduruluyor.
En iyi modelin ağırlıkları kaydediliyor.


In [8]:
result = test_model(model,test_loader,criterion,device)

Test Loss: 0.0999, Test Accuracy: 97.00%


In [9]:
array=np.array(result)
y_true = array[:, 0]
y_pred = array[:, 1]
cm = confusion_matrix(y_true, y_pred)
print(cm)

[[130   0]
 [  6  64]]


In [ ]:
neg = sum(cm[0])
pos = sum(cm[1])
pos_weight = torch.tensor([neg / pos]*1.5)
criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
criterion.to(device)

In [ ]:
train_model(model,epochs,optimizer,criterion, train_loader,val_loader,device,5)

In [17]:
resnet = create_resnet_model(num_classes=1, freeze_layers=False)
resnet.to(device)
checkpoint=torch.load("resnet_weights.pth", map_location=device)
resnet.load_state_dict(checkpoint['model_state_dict'])

densenet = create_densenet121_model(num_classes=1, freeze_layers=False)
densenet.to(device)
checkpoint=torch.load("densenet121_weights.pth", map_location=device)
densenet.load_state_dict(checkpoint['model_state_dict'])

efficient = create_efficientnet_model(num_classes=1, freeze_layers=False)
efficient.to(device)
checkpoint=torch.load("EfficientNet-B0_weights.pth", map_location=device)
efficient.load_state_dict(checkpoint['model_state_dict'])

efficient_tmp = create_efficientnet_model(num_classes=1, freeze_layers=False).to(device)
num_classes=1
embed_dim=512
num_layers=4
num_heads=8
dropout=0.4
last_model= HybridModel(base_model=efficient_tmp,num_classes=num_classes,embed_dim=embed_dim,num_layers=num_layers,num_heads=num_heads,dropout=dropout)
last_model.to(device)
checkpoint=torch.load("best_weights.pth", map_location=device)
last_model.load_state_dict(checkpoint['model_state_dict'])
criterion = torch.nn.BCEWithLogitsLoss()

In [22]:
result = test_model(resnet,test_loader,criterion,device)
array=np.array(result)
y_true = array[:, 0]
y_pred = array[:, 1]
cm = confusion_matrix(y_true, y_pred)
print("Resnet")
print(cm)
print(classification_report(y_true,y_pred))

result = test_model(densenet,test_loader,criterion,device)
array=np.array(result)
y_true = array[:, 0]
y_pred = array[:, 1]
cm = confusion_matrix(y_true, y_pred)
print("Densenet121")
print(cm)
print(classification_report(y_true,y_pred))

result = test_model(efficient,test_loader,criterion,device)
array=np.array(result)
y_true = array[:, 0]
y_pred = array[:, 1]
cm = confusion_matrix(y_true, y_pred)
print("EfficientNetB0")
print(cm)
print(classification_report(y_true,y_pred))

result = test_model(last_model,test_loader,criterion,device)
array=np.array(result)
y_true = array[:, 0]
y_pred = array[:, 1]
cm = confusion_matrix(y_true, y_pred)
print("Last model")
print(cm)
print(classification_report(y_true,y_pred))

result = test_model(last_model,test2_loader,criterion,device)
array=np.array(result)
y_true = array[:, 0]
y_pred = array[:, 1]
cm = confusion_matrix(y_true, y_pred)
print("Last model - Ekstra Test Veri Seti")
print(cm)
print(classification_report(y_true,y_pred))

Test Loss: 0.2006, Test Accuracy: 95.50%
Resnet
[[128   2]
 [  7  63]]
              precision    recall  f1-score   support

         0.0       0.95      0.98      0.97       130
         1.0       0.97      0.90      0.93        70

    accuracy                           0.95       200
   macro avg       0.96      0.94      0.95       200
weighted avg       0.96      0.95      0.95       200

Test Loss: 0.2482, Test Accuracy: 96.50%
Densenet121
[[130   0]
 [  7  63]]
              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97       130
         1.0       1.00      0.90      0.95        70

    accuracy                           0.96       200
   macro avg       0.97      0.95      0.96       200
weighted avg       0.97      0.96      0.96       200

Test Loss: 0.1987, Test Accuracy: 96.50%
EfficientNetB0
[[130   0]
 [  7  63]]
              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97       130
         1.

Eğitim sonuçlarında klasik cnn modellerinde densenet121 ile efficientnetb0 modelleri resnet50 modeline test başarısı metriğinde %1 fark atıp %96.5 e ulaşmıştır.
Densenet121 ve EfficientNetB0 modellerinden EfficientNetB0 ile farklı model ve modüllerle birleştirerek başarıyı arttırmayı hedeflendi.
CNN seçiminde başarılarının aynı olmasının yanında DenseNet121 modelinin eğitim süresi EfficientNetB0 a göre oldukça ağır kaldığı için EfficientNetB0 tercih edilmiştir.
Önce CNN modeline CBAM attention eklenmiş fakat beklenen iyileşme görülmemiştir.Daha sonra CNN modelinin sınıflandırma başlığı çıkartılıp ara lineer katman ile transformer encoder ile bağlanıp hibrit bir model oluşturulmuştur.Transformerı CNN den sonra koymamızın nedeni 1.si CNN den çıkan özelliklerde local bilgiler dahildir sadece transformers mimarisinde embeding yapmadığımız sürece konum bilgisi eksik kalır 2.olarak transformer ın CNN den çıkan özellikler birden çok bağlamda inceleyip daha detaylı bir çıkarım bulunabileceği düşünülmüştür.
Hibrit modelimizde ilk eğitim sonrasında 2 aşama ile öğrenme oranı düşürtülüp eğitim devam etmiştir.Bu aşamadan sonra karmaşıklık matrisinde fn leri düşürmek amacıyla criterionda dengesiz verilere dengeli tepki vermesi için pos_weight verilmiştir.2 aşamalı son eğitimin ardından fn sayısı düşürülüp fp sayısı artmıştır.Accuracy olarak baktığımızda büyük fark olmamış olsa da problemimizin sağlık verilerini işlediğini göz önünde bulunduracak olursak hasta olan kişilerin hasta değilsin teşhisinin hata boyutu daha büyük olduğunu söyliyebiliriz bu yüzden aslında bu önemli bir iyileştirme denilebilir.
Test sonuçlarımız tüm modellerimizin son aşamalarından elde edilip sınıflandırma raporları ile sunulmuştur.